# RMIA
From privacy meter

### Introduction

### Key objectives

### Environment steup with Pipenv

````
pipenv --python 3.12
````


In [1]:
# Clone the forked github repo
#!git clone https://github.com/privacytrustlab/ml_privacy_meter.git

In [2]:
# Change the directory to the cloned repo
# import sys
# sys.path.append('/content/ml_privacy_meter')

%cd ml_privacy_meter

/Users/tania.carvalho/Desktop/repos/AISec-class/ml_privacy_meter


/Users/tania.carvalho/.local/share/virtualenvs/AISec-class-DqnUEjF1/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
#!pipenv install datasets==2.21.0 transformers==4.44.2 torch==2.4.1 torchvision==0.19.1 torchaudio

Delete all CUDA-related lines from requirements.txt if your machine does not support it
```
nvidia-cublas-cu11
nvidia-cudnn-cu11
nvidia-...
triton
```

In [4]:
#!pip install -r requirements.txt

In [5]:
import math
import time

import pickle
import numpy as np
import torch
import yaml
from torch.utils.data import Subset

from audit import get_average_audit_results, audit_models, sample_auditing_dataset
from get_signals import get_model_signals
from models.utils import load_models, train_models, split_dataset_for_training
from util import (
    check_configs,
    setup_log,
    initialize_seeds,
    create_directories,
    load_dataset,
)

# Enable benchmark mode in cudnn to improve performance when input sizes are consistent
torch.backends.cudnn.benchmark = True

In [6]:
# Load cofigs
configs = "configs/heart-stat.yaml"
with open(configs, "rb") as f:
        configs = yaml.load(f, Loader=yaml.Loader)

# Validate configurations
check_configs(configs)

In [7]:
# Validate configurations
check_configs(configs)

# Initialize seeds for reproducibility
initialize_seeds(configs["run"]["random_seed"])

# Create necessary directories
log_dir = configs["run"]["log_dir"]
directories = {
    "log_dir": log_dir,
    "report_dir": f"{log_dir}/report",
    "signal_dir": f"{log_dir}/signals",
    "data_dir": "data",
}
create_directories(directories)

# Set up logger
logger = setup_log(
    directories["report_dir"], "time_analysis", configs["run"]["time_log"]
)

start_time = time.time()

### CIFAR Dataset

In [8]:
# baseline_time = time.time()
# dataset, population = load_dataset(configs, directories["data_dir"], logger)
# logger.info("Loading dataset took %0.5f seconds", time.time() - baseline_time)

In [9]:
# with open("data/cifar10.pkl", "rb") as f:
#     cifar10 = pickle.load(f)

# with open("data/cifar10_population.pkl", "rb") as f:
#     cifar10_pop = pickle.load(f)

# print("Number of images private data:", len(cifar10))
# print("Number of images population data:", len(cifar10_pop))


### Train models --- change cuda:o to cpu, and use CNN

In [10]:
# # Define experiment parameters
# num_experiments = configs["run"]["num_experiments"]
# num_reference_models = configs["audit"]["num_ref_models"]
# num_model_pairs = max(math.ceil(num_experiments / 2.0), num_reference_models + 1) # 2 model pairs = 4 models

# # train models
# baseline_time = time.time()

# # Split dataset for training two models per pair
# data_splits, memberships = split_dataset_for_training(
#     len(dataset), num_model_pairs
# )
# models_list = train_models(
#     log_dir, dataset, data_splits, memberships, configs, logger
# )
# logger.info(
#     "Model loading/training took %0.1f seconds", time.time() - baseline_time
# )

### Heart-statlog data

Details on dataset

In [11]:
import pandas as pd
import requests
from pathlib import Path

# Download dataset
url = "https://www.openml.org/data/download/6358/BNG_heart-statlog.csv"
csv_path = Path("heart-statlog.csv")

print("Downloading dataset...")
response = requests.get(url)
response.raise_for_status()

with open(csv_path, "wb") as f:
    f.write(response.content)

print("Download completed", csv_path.resolve())

# Parse file
columns = []
data_rows = []
data_section = False

with open(csv_path, "r") as f:
    for line in f:
        line = line.strip()

        # Extract column names from @attribute
        if line.lower().startswith("@attribute"):
            parts = line.split()
            col_name = parts[1]
            columns.append(col_name)

        # Start reading data
        elif line.lower() == "@data":
            data_section = True

        # Collect data rows
        elif data_section and line and not line.startswith("%"):
            # ARFF uses comma-separated values
            row = [x.strip() for x in line.split(",")]
            data_rows.append(row)

df = pd.DataFrame(data_rows, columns=columns)

# Save CSV
# df.to_csv(csv_path, index=False)
df.head()


Download completed /Users/tania.carvalho/Desktop/repos/AISec-class/ml_privacy_meter/heart-statlog.csv


,age,sex,chest,resting_blood_pressure,serum_cholestoral,fasting_blood_sugar,resting_electrocardiographic_results,maximum_heart_rate_achieved,exercise_induced_angina,oldpeak,slope,number_of_major_vessels,thal,class
0,53.494725,1,1.150395,117.978412,242.00937,0,0,133.361344,0,3.089391,2,1,3,present
1,37.320375,0,1.887693,118.45567,218.156844,1,2,148.458625,0,0,3,0,3,absent
2,48.520214,1,3,141.819366,173.382704,0,2,141.198191,0,1.071691,2,0,6,absent
3,59.587959,0,4,106.368725,222.732859,0,2,141.659888,1,0.866638,2,0,7,present
4,58.805677,1,3,121.035286,257.257441,0,0,145.333117,0,1.2126,3,0,7,absent


In [12]:
from dataset import TabularDataset
# Set up logger
logger = setup_log(
    directories["report_dir"], "time_analysis", configs["run"]["time_log"]
)

path = f"{configs["data"]["data_dir"]}/{configs["data"]["dataset"]}"

df["class"] = df["class"].apply(lambda x: 1 if x == "present" else 0)

df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)  # sample 10% of the dataset

df = df.to_numpy()
y = df[:, -1]
X = df[:, :-1].astype(np.float32)
training_size = int(
    len(y) * 0.75
)  # Splitting to create a population dataset
dataset = TabularDataset(X[:training_size], y[:training_size])
population = TabularDataset(X[training_size:], y[training_size:])
with open(f"{path}.pkl", "wb") as file:
    pickle.dump(dataset, file)
logger.info(f"Save data to {path}.pkl")
with open(f"{path}_population.pkl", "wb") as file:
    pickle.dump(population, file)
logger.info(f"Save population data to {path}_population.pkl")

2025-11-18 07:58:23,914 INFO     Save data to data/heart-statlog.pkl
2025-11-18 07:58:23,914 INFO     Save data to data/heart-statlog.pkl
2025-11-18 07:58:23,923 INFO     Save population data to data/heart-statlog_population.pkl
2025-11-18 07:58:23,923 INFO     Save population data to data/heart-statlog_population.pkl


Update ouput shape with the tested dataset [number of features, number of classes] -- models/utils.py

```
INPUT_OUTPUT_SHAPE = {
    "cifar10": [3, 10],
    "cifar100": [3, 100],
    "purchase100": [600, 100],
    "texas100": [6169, 100],
    "heart-statlog": [13, 2],
}
```

In [ ]:
print("Shape of array:", df[:, :-1].shape)       # number of rows/columns
print("Number of unique values in target column:", len(np.unique(df[:, -1])))

Shape of array: (100000, 13)
Number of unique values in last column: 2


In [14]:
# Define experiment parameters
num_experiments = configs["run"]["num_experiments"]
num_reference_models = configs["audit"]["num_ref_models"]
num_model_pairs = max(math.ceil(num_experiments / 2.0), num_reference_models + 1) # 2 model pairs = 4 models

# train models
baseline_time = time.time()

# Split dataset for training two models per pair
data_splits, memberships = split_dataset_for_training(
    len(dataset), num_model_pairs
)
models_list = train_models(
    log_dir, dataset, data_splits, memberships, configs, logger
)
logger.info(
    "Model loading/training took %0.1f seconds", time.time() - baseline_time
)

2025-11-18 07:58:23,951 INFO     Training 4 models
2025-11-18 07:58:23,951 INFO     Training 4 models
2025-11-18 07:58:23,957 INFO     --------------------------------------------------
2025-11-18 07:58:23,957 INFO     --------------------------------------------------
2025-11-18 07:58:23,960 INFO     Training model 0: Train size 37500, Test size 37500
2025-11-18 07:58:23,960 INFO     Training model 0: Train size 37500, Test size 37500


Using optimizer: SGD | Learning Rate: 0.1 | Weight Decay: 0
Epoch [1/100] | Train Loss: 0.6252 | Train Acc: 0.6632
Test Loss: 0.6045 | Test Acc: 0.6806
Epoch 1 took 16.36 seconds
Epoch [2/100] | Train Loss: 0.5938 | Train Acc: 0.6889
Test Loss: 0.5900 | Test Acc: 0.6902
Epoch 2 took 0.16 seconds
Epoch [3/100] | Train Loss: 0.5886 | Train Acc: 0.6921
Test Loss: 0.6022 | Test Acc: 0.6884
Epoch 3 took 0.15 seconds
Epoch [4/100] | Train Loss: 0.5883 | Train Acc: 0.6945
Test Loss: 0.5820 | Test Acc: 0.6959
Epoch 4 took 0.31 seconds
Epoch [5/100] | Train Loss: 0.5815 | Train Acc: 0.6979
Test Loss: 0.5900 | Test Acc: 0.6931
Epoch 5 took 0.14 seconds
Epoch [6/100] | Train Loss: 0.5824 | Train Acc: 0.6945
Test Loss: 0.5831 | Test Acc: 0.6903
Epoch 6 took 0.15 seconds
Epoch [7/100] | Train Loss: 0.5874 | Train Acc: 0.6920
Test Loss: 0.5699 | Test Acc: 0.7057
Epoch 7 took 0.18 seconds
Epoch [8/100] | Train Loss: 0.5834 | Train Acc: 0.6916
Test Loss: 0.5768 | Test Acc: 0.6952
Epoch 8 took 0.16 sec

2025-11-18 07:58:55,774 INFO     Train accuracy 0.8139733333333333, Train Loss 0.40638875413914116
2025-11-18 07:58:55,774 INFO     Train accuracy 0.8139733333333333, Train Loss 0.40638875413914116
2025-11-18 07:58:55,776 INFO     Test accuracy 0.8128266666666667, Test Loss 0.41102946818280384
2025-11-18 07:58:55,776 INFO     Test accuracy 0.8128266666666667, Test Loss 0.41102946818280384
2025-11-18 07:58:55,778 INFO     Training model 0 took 31.820865154266357 seconds
2025-11-18 07:58:55,778 INFO     Training model 0 took 31.820865154266357 seconds
2025-11-18 07:58:55,781 INFO     --------------------------------------------------
2025-11-18 07:58:55,781 INFO     --------------------------------------------------
2025-11-18 07:58:55,782 INFO     Training model 1: Train size 37500, Test size 37500
2025-11-18 07:58:55,782 INFO     Training model 1: Train size 37500, Test size 37500


Using optimizer: SGD | Learning Rate: 0.1 | Weight Decay: 0
Epoch [1/100] | Train Loss: 0.6460 | Train Acc: 0.6317
Test Loss: 0.6146 | Test Acc: 0.6674
Epoch 1 took 16.17 seconds
Epoch [2/100] | Train Loss: 0.6082 | Train Acc: 0.6692
Test Loss: 0.5968 | Test Acc: 0.6869
Epoch 2 took 0.14 seconds
Epoch [3/100] | Train Loss: 0.6022 | Train Acc: 0.6750
Test Loss: 0.6004 | Test Acc: 0.6823
Epoch 3 took 0.15 seconds
Epoch [4/100] | Train Loss: 0.5998 | Train Acc: 0.6756
Test Loss: 0.6229 | Test Acc: 0.6701
Epoch 4 took 0.28 seconds
Epoch [5/100] | Train Loss: 0.5982 | Train Acc: 0.6817
Test Loss: 0.6167 | Test Acc: 0.6558
Epoch 5 took 0.13 seconds
Epoch [6/100] | Train Loss: 0.5984 | Train Acc: 0.6782
Test Loss: 0.5969 | Test Acc: 0.6850
Epoch 6 took 0.13 seconds
Epoch [7/100] | Train Loss: 0.5937 | Train Acc: 0.6862
Test Loss: 0.5838 | Test Acc: 0.7026
Epoch 7 took 0.14 seconds
Epoch [8/100] | Train Loss: 0.5934 | Train Acc: 0.6866
Test Loss: 0.5905 | Test Acc: 0.6806
Epoch 8 took 0.13 sec

2025-11-18 08:00:06,986 INFO     Train accuracy 0.72792, Train Loss 0.5028683516849466
2025-11-18 08:00:06,986 INFO     Train accuracy 0.72792, Train Loss 0.5028683516849466
2025-11-18 08:00:06,987 INFO     Test accuracy 0.7301333333333333, Test Loss 0.5024171599320003
2025-11-18 08:00:06,987 INFO     Test accuracy 0.7301333333333333, Test Loss 0.5024171599320003
2025-11-18 08:00:06,988 INFO     Training model 1 took 71.20666480064392 seconds
2025-11-18 08:00:06,988 INFO     Training model 1 took 71.20666480064392 seconds
2025-11-18 08:00:06,993 INFO     --------------------------------------------------
2025-11-18 08:00:06,993 INFO     --------------------------------------------------
2025-11-18 08:00:06,993 INFO     Training model 2: Train size 37500, Test size 37500
2025-11-18 08:00:06,993 INFO     Training model 2: Train size 37500, Test size 37500


...

2025-11-18 08:01:18,130 INFO     Train accuracy 0.67592, Train Loss 0.5441305444759577
2025-11-18 08:01:18,130 INFO     Train accuracy 0.67592, Train Loss 0.5441305444759577
2025-11-18 08:01:18,131 INFO     Test accuracy 0.67376, Test Loss 0.5495240943772453
2025-11-18 08:01:18,131 INFO     Test accuracy 0.67376, Test Loss 0.5495240943772453
2025-11-18 08:01:18,132 INFO     Training model 2 took 71.13932776451111 seconds
2025-11-18 08:01:18,132 INFO     Training model 2 took 71.13932776451111 seconds
2025-11-18 08:01:18,139 INFO     --------------------------------------------------
2025-11-18 08:01:18,139 INFO     --------------------------------------------------
2025-11-18 08:01:18,140 INFO     Training model 3: Train size 37500, Test size 37500
2025-11-18 08:01:18,140 INFO     Training model 3: Train size 37500, Test size 37500


Epoch [100/100] | Train Loss: 0.4525 | Train Acc: 0.7911
Test Loss: 0.5495 | Test Acc: 0.6738
Epoch 100 took 0.14 seconds
Using optimizer: SGD | Learning Rate: 0.1 | Weight Decay: 0
Epoch [1/100] | Train Loss: 0.6558 | Train Acc: 0.6183
Test Loss: 0.6147 | Test Acc: 0.6691
Epoch 1 took 15.87 seconds
Epoch [2/100] | Train Loss: 0.6121 | Train Acc: 0.6741
Test Loss: 0.6037 | Test Acc: 0.6685
Epoch 2 took 0.16 seconds
Epoch [3/100] | Train Loss: 0.6059 | Train Acc: 0.6766
Test Loss: 0.6233 | Test Acc: 0.6414
Epoch 3 took 0.18 seconds
Epoch [4/100] | Train Loss: 0.6016 | Train Acc: 0.6795
Test Loss: 0.6153 | Test Acc: 0.6641
Epoch 4 took 0.34 seconds
Epoch [5/100] | Train Loss: 0.5985 | Train Acc: 0.6810
Test Loss: 0.5749 | Test Acc: 0.7051
Epoch 5 took 0.14 seconds
Epoch [6/100] | Train Loss: 0.5964 | Train Acc: 0.6830
Test Loss: 0.5828 | Test Acc: 0.7016
Epoch 6 took 0.14 seconds
Epoch [7/100] | Train Loss: 0.5909 | Train Acc: 0.6873
Test Loss: 0.5748 | Test Acc: 0.7012
Epoch 7 took 0.14

2025-11-18 08:02:28,775 INFO     Train accuracy 0.7630133333333333, Train Loss 0.4789587985090658
2025-11-18 08:02:28,775 INFO     Train accuracy 0.7630133333333333, Train Loss 0.4789587985090658
2025-11-18 08:02:28,776 INFO     Test accuracy 0.7702666666666667, Test Loss 0.4742466815474893
2025-11-18 08:02:28,776 INFO     Test accuracy 0.7702666666666667, Test Loss 0.4742466815474893
2025-11-18 08:02:28,777 INFO     Training model 3 took 70.63755393028259 seconds
2025-11-18 08:02:28,777 INFO     Training model 3 took 70.63755393028259 seconds


Epoch [100/100] | Train Loss: 0.4752 | Train Acc: 0.7744
Test Loss: 0.4742 | Test Acc: 0.7703
Epoch 100 took 0.13 seconds


2025-11-18 08:03:08,802 INFO     Model loading/training took 284.9 seconds
2025-11-18 08:03:08,802 INFO     Model loading/training took 284.9 seconds


### Auditing dataset

In [15]:
auditing_dataset, auditing_membership = sample_auditing_dataset(
        configs, dataset, logger, memberships
    )

# Also downsample the population set size if specified in the config
population = Subset(
    population,
    np.random.choice(
        len(population),
        configs["audit"].get("population_size", len(population)),
        replace=False,
    ),
)

### Compute signals

In [16]:
baseline_time = time.time()
signals = get_model_signals(models_list, auditing_dataset, configs, logger)
population_signals = get_model_signals(
        models_list, population, configs, logger, is_population=True
    )
logger.info("Preparing signals took %0.5f seconds", time.time() - baseline_time)

2025-11-18 08:03:08,827 WARNING  Signals shape (750000) does not match the expected size (75000). This mismatch is likely due to a change in the training data size.
2025-11-18 08:03:08,827 WARNING  Signals shape (750000) does not match the expected size (75000). This mismatch is likely due to a change in the training data size.
2025-11-18 08:03:08,827 INFO     Ignoring the signals on disk and recomputing.
2025-11-18 08:03:08,827 INFO     Ignoring the signals on disk and recomputing.
2025-11-18 08:03:37,053 INFO     Computing signals for all models.
2025-11-18 08:03:37,053 INFO     Computing signals for all models.
Computing softmax: 100%|██████████| 15/15 [00:00<00:00, 1124.44it/s]
2025-11-18 08:03:37,146 INFO     Signals saved to disk.
2025-11-18 08:03:37,146 INFO     Signals saved to disk.
2025-11-18 08:03:37,151 WARNING  Signals shape (250000) does not match the expected size (25000). This mismatch is likely due to a change in the training data size.
2025-11-18 08:03:37,151 WARNING 

In [19]:
# Perform the privacy audit
baseline_time = time.time()
target_model_indices = list(range(num_experiments))
mia_score_list, membership_list = audit_models(
        f"{directories['report_dir']}/exp",
        target_model_indices,
        signals,
        population_signals,
        auditing_membership,
        num_reference_models,
        logger,
        configs,
    )

if len(target_model_indices) > 1:
    logger.info(
        "Auditing privacy risk took %0.1f seconds", time.time() - baseline_time
    )

# Get average audit results across all experiments
if len(target_model_indices) > 1:
    get_average_audit_results(
        directories["report_dir"], mia_score_list, membership_list, logger
    )

logger.info("Total runtime: %0.5f seconds", time.time() - start_time)

2025-11-18 08:06:09,951 INFO     Fine-tuning offline_a using paired model 1
2025-11-18 08:06:09,951 INFO     Fine-tuning offline_a using paired model 1
2025-11-18 08:06:15,071 INFO     offline_a=0.00: AUC 0.5013
2025-11-18 08:06:15,071 INFO     offline_a=0.00: AUC 0.5013
2025-11-18 08:06:17,017 INFO     offline_a=0.10: AUC 0.5014
2025-11-18 08:06:17,017 INFO     offline_a=0.10: AUC 0.5014
2025-11-18 08:06:19,646 INFO     offline_a=0.20: AUC 0.5014
2025-11-18 08:06:19,646 INFO     offline_a=0.20: AUC 0.5014
2025-11-18 08:06:21,018 INFO     offline_a=0.30: AUC 0.5014
2025-11-18 08:06:21,018 INFO     offline_a=0.30: AUC 0.5014
2025-11-18 08:06:22,360 INFO     offline_a=0.40: AUC 0.5012
2025-11-18 08:06:22,360 INFO     offline_a=0.40: AUC 0.5012
2025-11-18 08:06:23,769 INFO     offline_a=0.50: AUC 0.5012
2025-11-18 08:06:23,769 INFO     offline_a=0.50: AUC 0.5012
2025-11-18 08:06:26,251 INFO     offline_a=0.60: AUC 0.5013
2025-11-18 08:06:26,251 INFO     offline_a=0.60: AUC 0.5013
2025-11-

<Figure size 640x480 with 0 Axes>

### Tasks

1) Experiment with a different dataset.
2) Implement a regularization technique as a defense strategy against MIAs.
3) Re-evaluate MIAs.